# Bedrock Knowledge Base Retrieval and Generation with Guardrails

## Description
This notebook demonstrates how to enhance a Retrieval-Augmented Generation (RAG) pipeline using Amazon Bedrock with Guardrails for better model control and filtering. We will walk through retrieving data from a knowledge base, applying guardrails to control the generation of responses, and filtering results using metadata.

![Guardrails](./guardrail.png)

## 1: Import Required Functions

In [1]:
# Import necessary functions from advanced_rag_utils
from advanced_rag_utils import (
    load_variables,
    get_model_arn,
    get_guardrail_arn,
    setup_bedrock_client,
    create_standard_filter,
    retrieve_and_generate_with_conditional_guardrails,
    display_markdown_result,
    extract_text_response
)

## 2: Load Configuration Variables

In [2]:
# Load the configuration variables
variables = load_variables("../variables.json")
variables

{'accountNumber': '977018081517',
 'regionName': 'us-west-2',
 'collectionArn': 'arn:aws:aoss:us-west-2:977018081517:collection/x6j41w4wldt22s0mc4oa',
 'collectionId': 'x6j41w4wldt22s0mc4oa',
 'vectorIndexName': 'ws-index-',
 'bedrockExecutionRoleArn': 'arn:aws:iam::977018081517:role/advanced-rag-workshop-bedrock_execution_role-us-west-2',
 's3Bucket': '977018081517-us-west-2-advanced-rag-workshop',
 'kbFixedChunk': 'WFJRA66ARJ',
 'kbSemanticChunk': 'XRQUIKPQAB',
 'kbHierarchicalChunk': 'DBCDV6VOVB',
 'kbCustomChunk': 'SVUWGDQTQZ',
 'sagemakerLLMEndpoint': 'endpoint-llama-3-2-3b-instruct-2025-05-01-17-04-39',
 'guardrail_id': 'vcd1cw90tu5w',
 'guardrail_version': '1'}

## 3: Define ARN and Configuration Details

In [3]:
# Setting up configuration for Bedrock and Guardrails
account_number = variables['accountNumber']
knowledge_base_id = variables['kbFixedChunk']   
model_id = 'us.amazon.nova-pro-v1:0' 
guardrail_version = variables['guardrail_version'] 
guardrail_id = variables['guardrail_id']    

# Get ARNs for the model and guardrails using utility functions
model_arn = get_model_arn(account_number, model_id)
guardrail_arn = get_guardrail_arn(account_number, guardrail_id)

## 4: Set Up Bedrock Client

In [4]:
# Initialize the Bedrock agent runtime client
bedrock_client = setup_bedrock_client("us-west-2")

## 5: Define Metadata Filter

In [5]:
# Create a standard filter for document type and year
metadata_filter = create_standard_filter('10K Report', 2023)

## 6: Test Guardrails for Investment Advice
Let's ask the model for investment advice. When we created the guardrails, we restricted Bedrock from providing any investment advice. Bedrock should return a preconfigured response "This request cannot be processed due to safety protocols"

In [6]:
# Define the query that will be sent to the model
query = "based on your amazon's results should I buy amazon stock?"

### 6.1: Response Without Guardrails

In [7]:
# Get response without applying guardrails
response_without_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    bedrock_client=bedrock_client,
    metadata_filter=metadata_filter,
    use_guardrails=False
)

# Display the response in Markdown format
display_markdown_result(response_without_guardrails)

It's important to note that making investment decisions should be based on thorough research, analysis, and consideration of various factors, including your own financial goals, risk tolerance, and investment horizon. While I can provide information and analysis based on the given passages, the decision to buy Amazon stock ultimately depends on your individual circumstances and investment strategy.

That being said, let's analyze the information provided in the passages to gain some insights:

1. **Forward-looking Statements and Risks** (Passage %[1]%):
   - Amazon acknowledges that forward-looking statements are inherently uncertain and that actual results could differ materially due to a variety of factors. These include fluctuations in foreign exchange rates, global economic conditions, customer demand, inflation, interest rates, supply chain constraints, world events, competition, and more.
   - The passage highlights the importance of considering the "Risk Factors" section (Item 1A) for a detailed understanding of the risks and uncertainties that could impact Amazon's performance.

2. **Market Value and Outstanding Shares** (Passage %[2]%):
   - As of June 30, 2022, the aggregate market value of voting stock held by non-affiliates was approximately \$944.7 billion.
   - The number of shares of common stock outstanding as of January 25, 2023, was 10,247,259,757.

3. **Financial Statements** (Passage %[3]%):
   - The consolidated statements of stockholders’ equity show changes in common stock, treasury stock, additional paid-in capital, accumulated other comprehensive income (loss), and retained earnings over the years.
   - Net income for the fiscal year ended December 31, 2022, was \$21,331 million, with basic and diluted earnings per share of \$2.13 and \$2.09, respectively.

4. **Business Description and Accounting Policies** (Passage %[4]%):
   - Amazon aims to be Earth’s most customer-centric company, serving various customer sets including consumers, sellers, developers, enterprises, content creators, advertisers, and employees.
   - The company focuses on selection, price, and convenience for consumers and offers programs to enable sellers and content creators to grow their businesses.

5. **Financial Performance** (Passage %[5]%):
   - The passage provides details on net income, earnings per share, and weighted-average shares used in the computation of earnings per share for different fiscal years.

### Considerations for Investing in Amazon Stock:

- **Financial Performance**: Amazon has shown strong financial performance with significant net income and positive earnings per share. However, past performance is not indicative of future results.

- **Market Position**: Amazon is a dominant player in e-commerce, cloud computing (AWS), and various other sectors. Its strong market position can be a positive indicator.

- **Risks and Uncertainties**: As highlighted in Passage %[1]%, there are numerous risks and uncertainties that could impact Amazon's future performance. Investors should carefully review the "Risk Factors" section for a comprehensive understanding.

- **Market Conditions**: The performance of Amazon stock can be influenced by broader market conditions, economic factors, and geopolitical events.

- **Valuation**: The high market value and the number of outstanding shares indicate that Amazon is a large-cap stock. Investors should consider whether the current valuation aligns with their investment thesis.

### Conclusion:
While Amazon has demonstrated strong financial performance and holds a significant market position, investing in its stock involves risks. It is crucial to conduct thorough research, consider your own financial situation, and possibly consult with a financial advisor before making any investment decisions.

### 6.2: Response With Guardrails

In [8]:
# Get response with guardrails applied
response_with_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    bedrock_client=bedrock_client,
    metadata_filter=metadata_filter,
    use_guardrails=True,
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)

# Extract and print the text response
print(extract_text_response(response_with_guardrails))

I can only process questions related to AWS services and Amazon 10K filings.


## 7. Test Guardrails for PII Data

In [9]:
# Define a new query about Amazon's CFO
query="Who is the current CFO of Amazon?"

### 7.1: Response Without Guardrails

In [10]:
# Get response without applying guardrails
response_without_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    bedrock_client=bedrock_client,
    metadata_filter=metadata_filter,
    use_guardrails=False
)

# Extract and print the text response
print(extract_text_response(response_without_guardrails))

Based on the context provided in the passages, the current Chief Financial Officer (CFO) of Amazon is Brian T. Olsavsky. This is explicitly mentioned in Passage %[3]%, where it states, "Brian T. Olsavsky. Mr. Olsavsky has served as Senior Vice President and Chief Financial Officer since June 2015..." Additionally, Passage %[4]% also lists him as the "Senior Vice President and Chief Financial Officer" under the section about Executive Officers.


### 7.2: Response With Guardrails (PII Anonymization)

In [11]:
# Get response with guardrails applied
response_with_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    bedrock_client=bedrock_client,
    metadata_filter=metadata_filter,
    use_guardrails=True,
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)

# Extract and print the text response
print(extract_text_response(response_with_guardrails))

Based on the context provided in the passages, the current Chief Financial Officer (CFO) of Amazon is {NAME}. This is explicitly mentioned in Passage %[3]% and Passage %[4]%, where it states:

- Passage %[3]%: "{NAME}. Mr. {NAME} has served as Senior Vice President and Chief Financial Officer since June 2015..."
- Passage %[4]%: "{NAME} 59 Senior Vice President and Chief Financial Officer"

Therefore, the current CFO of Amazon is {NAME}.
